<div align="right"><i>Peter Norvig<br>2012<br>updated 2019, 2023</i></div>

# Weighing Twelve Balls on a Scale: ①②③④ ⚖ ⑤⑥⑦⑧ 

Here is [a classic](https://en.wikipedia.org/wiki/Balance_puzzle) brain-teaser puzzle (first appearing in 1945), meant to be solved with paper and pencil:  

- *You are given twelve identical-looking balls and a two-sided balance scale. One of the balls is of a different weight, although you don't know whether it is lighter or heavier. How can you use just three weighings of the scale to determine not only what the different ball is, but also whether it is lighter or heavier?*



However I want to solve not just this specific puzzle, but related puzzles where you can vary (a) the number of balls, (b) the number of weighings allowed, and (c) whether the different ball is known to be heavier, lighter, either, or possibly neither. So I'll be better served by a computer program, not a pencil. 

I originally wrote this program in Lisp around 1980, ported it to Python in 2012, and am republishing it here in revised form after I saw the puzzle was mentioned in the [538 Riddler](https://fivethirtyeight.com/features/which-billiard-ball-is-rigged/) for 16 August 2019. It also appeared in a [Numberplay](https://wordplay.blogs.nytimes.com/2014/07/21/12coin/) column in 2014 (with coins instead of balls), in the Museum of Math's [Mind-Benders for the Quarantined!](https://momath.org/civicrm/?page=CiviCRM&q=civicrm/event/info&reset=1&id=1620) series in 2020, and in many other venues.


# Defining the Vocabulary
    
Here are the main concepts:

- **Ball**: A ball is represented by a positive integer, just to tell them apart. A list of balls by a list, e.g. `[1, 2, 3]`.
- **Oddball**: One of the balls is different in weight; I'll use the term **oddball** to indicate which ball is different, and how. I'll use `+N` to mean ball `N` is heavier, `-N` to mean ball `N` is lighter, and `0` to mean that all the balls weigh the same. (So `0` cannot be a ball number.) 
- **Puzzle**: I'll express the puzzle stated above with `Puzzle(N=12, weighings=3)`. A puzzle is defined by the number of balls, the number of weighings allowed, and the possible oddballs (I'll say how to describe them later).
- **Weighing**: I can weigh a collection of balls on the left versus a collection on the right, and the result will be that the left side is greater than, equal to, or less than the right in weight. I'll denote that with the call `weigh(L, R, oddball)`, which returns a string, `'gt'`, `'eq'`, or `'lt'`.
- **Solution**: Given a puzzle, a solution is a **strategy tree** that can correctly discover the oddball, whatever it is, in the allowable number of weighings. `None` indicates the failure to find a solution.
- **Strategy tree**: A tree whose leaf nodes are oddballs and whose interior nodes have 5 components:  `Node(L, R, gt, eq, lt)` is a node where `L` and `R` are the lists of balls to be weighed, and `gt`, `eq`, and `lt` are branches for subtrees corresponding to the three possible weighing results. 
- **Following a path**: I'll use `follow(tree, oddball)` to say *follow the path through the tree, doing each weighing under the assumption of the given oddball, and return the leaf node reached by the path–the oddball that the tree predicts.* Note that the function `follow` gets to see what the oddball is, but the `tree` does not; there is one static tree that has to figure out the oddball by doing weighings, and it has to work for any oddball.
- **Valid solution**: A solution, `tree`, is a **valid solution** to a puzzle if`follow(tree, oddball) == oddball` for every oddball in the puzzle. I use `valid(tree, puzzle)` for this.


# Implementation of Data Types

Most of the types are straightforward. The `Puzzle(...)` class is more complex. You can specify the number of balls, the number of allowable weighings, and the possible oddballs, which should be a set consisting of oddball numbers, or it could contain `lt` and/or `gt` to indicate that any of the balls might be less than or greater than the others, respectively.

In [1]:
from collections import namedtuple
from typing import Union, Literal, Optional, Dict, Set, Tuple, List
import random

Ball    = int      # Balls are positive integers
Oddball = int      # Oddballs are positive, negative, or zero integers
Leaf    = Oddball  # Leaves in a tree are oddballs
Node    = namedtuple('Node', 'L, R, gt, eq, lt') # A Node is a tuple of 5 components
Tree    = Union[Node, Leaf]   # A tree can be an interior node or a leaf
results = (lt, eq, gt) = ('lt', 'eq', 'gt') # The result of a weighing is one of these three

class Puzzle:
    """Represent a specific ball-weighing puzzle. You can specify the number of balls, `N`,
    the number of allowable `weighings`, and the `oddballs` as a set of oddball numbers, and/or
    the constants `lt` or `gt`."""
    def __init__(self, N=12, weighings=3, oddballs={lt, gt}):
        self.ob_input = oddballs # Remember the original input
        balls = tuple(range(1, N + 1))
        if gt in oddballs: oddballs = oddballs - {gt} | {+b for b in balls}
        if lt in oddballs: oddballs = oddballs - {lt} | {-b for b in balls}
        if eq in oddballs: oddballs = oddballs - {eq} | {0}
        self.__dict__.update(N=N, weighings=weighings, oddballs=oddballs, balls=balls)
    def __repr__(self): 
        return f'Puzzle(N={self.N}, weighings={self.weighings}, oddballs={self.ob_input}'

# Implementation of Basic Functions

Here are all the straightforward utility functions; everything except the main function `find_tree` that actually solves a puzzle.

In [2]:
def weigh(L, R, oddball) -> Literal[results]:
    "Weigh balls L against balls R, given the oddball; return gt, eq, or lt."
    Δ = sum(weight(b, oddball) for b in L) - sum(weight(b, oddball) for b in R)
    return gt if Δ > 0 else lt if Δ < 0 else eq

def weight(ball, oddball) -> int: 
    "How heavy is this ball (given that we know what the oddball is)?"
    return 101 if +ball == oddball else 99 if -ball == oddball else 100
    
def solve(puzzle) -> Optional[Tree]:
    "Return a valid tree (one that solves the puzzle), or None for failure."
    tree = find_tree(puzzle, puzzle.oddballs, puzzle.weighings)
    return tree if valid(tree, puzzle) else None
    
def follow(tree, oddball) -> Oddball:
    "Follow a path through the tree and return the oddball that the tree leads us to."
    if isinstance(tree, Leaf):
        return tree
    else:
        branch = weigh(tree.L, tree.R, oddball)
        return follow(getattr(tree, branch), oddball)
    
def valid(tree, puzzle) -> bool:
    "Does the strategy tree solve the puzzle correctly for all possible oddballs?"
    return (tree is not None and
            depth(tree) <= puzzle.weighings and 
            all(follow(tree, oddball) == oddball 
                for oddball in puzzle.oddballs))

def depth(tree) -> int:
    "Maximum depth (number of weighings) in tree."
    return (0 if isinstance(tree, Leaf) or tree is None
            else 1 + max(depth(tree.gt), depth(tree.eq), depth(tree.lt)))

def first(iterable): return next(iter(iterable))

Let's try out some of these functions:

In [3]:
# A puzzle with 8 balls and 3 weighings allowed
p8 = Puzzle(8, 3) 
p8

Puzzle(N=8, weighings=3, oddballs={'gt', 'lt'}

In [4]:
p8.balls 

(1, 2, 3, 4, 5, 6, 7, 8)

In [5]:
p8.oddballs

{-8, -7, -6, -5, -4, -3, -2, -1, 1, 2, 3, 4, 5, 6, 7, 8}

In [6]:
# If we weigh balls [1, 2] against [3, 4] and 4 is lighter, the result should be 'gt'
weigh([1, 2], [3, 4], -4)

'gt'

In [7]:
# If ball 4 is light, then it should be true that ball 1 weighhs more than ball 4
weight(1, oddball=-4) > weight(4, oddball=-4)

True

# Finding a Valid Solution Tree

Now we've got all the pieces we need to find a valid tree to solve a puzzle. The key concept is that a **weighing** gives us information by making a **partition** of the possible **oddballs** into branches for each of the three possible weighing results: `gt`, `eq`, or `lt`. If each of the partitions is small enough, subsequent weighings can handle each of them. Here's what I mean by a partition:

In [8]:
Partition = Dict[Literal[results], Set[Oddball]]

def partition(L, R, oddballs) -> Partition:
    "A dict that partitions the possible outcomes from weighing L versus R."
    dic = dict(gt=set(), eq=set(), lt=set())
    for odd in oddballs:
        dic[weigh(L, R, odd)].add(odd)
    return dic

For example, with 12 balls, if we weigh balls [1, 2, 3] on the left versus [10, 11, 12] on the right, then there are 6 ways  the left side can be greater than the right: either 10, 11 or 12 is lighter or  1, 2, or 3 is heavier. Similarly there are 6 ways of getting a `lt` weighing result. The remaining 12 possibilities&mdash;balls 4 through 9 being either heavier or lighter&mdash;show up in the `eq` branch of the partition:

In [9]:
p12 = Puzzle(12, 3)

partition([1, 2, 3], [10, 11, 12], p12.oddballs)

{'gt': {-12, -11, -10, 1, 2, 3},
 'eq': {-9, -8, -7, -6, -5, -4, 4, 5, 6, 7, 8, 9},
 'lt': {-3, -2, -1, 10, 11, 12}}

If this was the first weighing in our strategy tree, could we go on to solve the puzzle in 3 weighings? 

**No!** The key insight is that a tree with <i>w</i> weighings can distinguish at most  3<sup><i>w</i></sup> oddballs, because each weighing has three possible results. After using one weighing we can only solve the puzzle if every branch of the partition has no more than 3<sup>2</sup> = 9 oddballs, but here we have 12 oddballs in the `eq` branch. We call this a **bad partition**. A good partition is one where no branch has more than 3<sup><i>w</i></sup> oddballs. Being a good partition does not guarantee that the puzzle is solvable from there; but being a bad partition guarantees that it is not solvable.
 

In [10]:
def good_partition(part, w) -> bool:
    "Is this a partition which might lead to a solution in w more weighings?"
    return all(len(oddballs) <= 3 ** w for oddballs in part.values())

The following is a good partition because each of the branches has 8 possibilities, and 8 is less than 3<sup>2</sup>:

In [11]:
p = partition([1, 2, 3, 4], [9, 10, 11, 12], p12.oddballs)
p

{'gt': {-12, -11, -10, -9, 1, 2, 3, 4},
 'eq': {-8, -7, -6, -5, 5, 6, 7, 8},
 'lt': {-4, -3, -2, -1, 9, 10, 11, 12}}

In [12]:
good_partition(p, 2)

True

So now we have a viable approach to implementing `find_tree`:

   - We call `find_tree(puzzle, oddballs, weighings)`. At the top level, the oddballs and number of weighings come from the puzzle. At recursive levels, we use the oddballs from the partition.
   - We handle trivial cases when there are zero or one oddballs remaining, or no weighings remaining.
   - Otherwise, we ask `good_partitions` to generate random partitions and yield the good ones.
   - Given a good partition, try to find valid trees for each of the three branches. If we can find that, return the tree as a solution.
   - If we can't find good trees for any partition, return `None` to indicate failure.
   
The rest of the work is done by `good_partitions`:

   - Select two lists of balls to be weighed, `L` and `R`, both of length `B`.
   - See what partition `L` and `R` gives us.
   - If it is good, yield the `(L, R, partition)` tuple.
   - Repeat for all possible values of `B`.
   - Randomly shuffle the balls and repeat the whole process, trying to generate more good partitions. I chose to repeat 100 times, to have a good chance of finding a solution for solvable puzzles, and of terminating in just a few seconds for unsolvable puzzles. 
   - However, I note that for most puzzles, on the first weighing, there's no sense making multiple tries with randomly shuffled  balls–the only choice that matters is how many balls, `B`, to put on each side, because the balls are undifferentiated at this point. So I try each value of `B` only once on the first weighing. However, some puzzles actually do differentiate between balls (you could use `oddballs` to say that ball 1 can only be light and ball 2 can only be heavy); for those puzzles we need to try repeatedly.
   - If I wanted to solve puzzles with thousands of balls, not just dozens, I would come back and improve `good_partitions`.

In [13]:
def find_tree(puzzle, oddballs, weighings) -> Tree or None:
    "Find a tree that covers all the oddballs in the given number of weighings."
    if len(oddballs) == 1:
        return first(oddballs) # One possibility left; we're done; leaf node
    elif len(oddballs) == 0:
        return 0               # No oddball (a possibility for some puzzles)
    elif weighings == 0:
        return None            # No solution; failure
    else:
        for L, R, part in good_partitions(puzzle, oddballs, weighings - 1):
            subtrees = {r: find_tree(puzzle, part[r], weighings - 1) for r in part}
            if None not in subtrees.values(): 
                return Node(L, R, **subtrees)
    
def good_partitions(puzzle, oddballs, weighings, tries=100) -> Tuple[List[Ball], List[Ball], Partition]:
    "Yield random (L, R, partition) tuples of good partitions."
    balls = sorted(puzzle.balls)
    if weighings == puzzle.weighings and puzzle.ob_input.issubset({eq, lt, gt, 0}): 
        tries = 1 # First weighing, undifferentiated balls: only need to try once.
    for _ in range(tries): 
        for B in range(1, 1 + len(balls) // 2):
            L, R = balls[:B], balls[-B:]
            dic = partition(L, R, oddballs)
            if good_partition(dic, weighings):
                yield L, R, dic
        random.shuffle(balls)

Here we see that  `good_partitions` does its job:

In [14]:
first(good_partitions(p12, p12.oddballs, 2))

([1, 2, 3, 4],
 [9, 10, 11, 12],
 {'gt': {-12, -11, -10, -9, 1, 2, 3, 4},
  'eq': {-8, -7, -6, -5, 5, 6, 7, 8},
  'lt': {-4, -3, -2, -1, 9, 10, 11, 12}})

# Solving Some Puzzles

Now we're ready to solve puzzles! We'll start with the original 12-ball puzzle, `p12`:

In [15]:
solve(p12)

Node(L=[1, 2, 3, 4], R=[9, 10, 11, 12], gt=Node(L=[9, 2, 11], R=[4, 10, 8], gt=Node(L=[1, 2], R=[11, 12], gt=2, eq=-10, lt=0), eq=Node(L=[4, 3], R=[9, 1], gt=3, eq=-12, lt=1), lt=Node(L=[5, 12], R=[11, 4], gt=-11, eq=-9, lt=4)), eq=Node(L=[7, 10, 2, 4, 11], R=[8, 12, 5, 9, 1], gt=Node(L=[1, 2, 3, 4, 5], R=[8, 9, 10, 11, 12], gt=-8, eq=7, lt=-5), eq=Node(L=[1, 2, 3, 4, 5, 6], R=[7, 8, 9, 10, 11, 12], gt=6, eq=0, lt=-6), lt=Node(L=[1, 2, 3, 4, 5], R=[8, 9, 10, 11, 12], gt=5, eq=-7, lt=8)), lt=Node(L=[5, 2, 4, 9], R=[8, 1, 10, 6], gt=Node(L=[1], R=[12], gt=0, eq=9, lt=-1), eq=Node(L=[7, 2, 8], R=[3, 12, 9], gt=-3, eq=11, lt=12), lt=Node(L=[12, 1, 4], R=[11, 2, 6], gt=-2, eq=10, lt=-4)))

OK, that's hard to read&mdash;my bad. Let's look at a much easier puzzle: 3 balls, 1 weighing allowed, and the oddball can only be heavier:

In [16]:
tree = solve(Puzzle(3, 1, oddballs={gt}))
tree

Node(L=[1], R=[3], gt=1, eq=2, lt=3)

This trivial tree says you weigh the first ball against the third (leaving the second unweighed), and the three possible weighing results tell you which of the three balls  is heavier. 

# Prettier Output

Let's make the output easier to read. The function `do(puzzle)` will solve a puzzle, verify that the tree is valis, and print the tree in a pretty format.

I'll use `①②③④ ⚖ ⑨⑩⑪⑫ ➔` to represent a weighing; I'll use `>:+①` to represent the situation where the left side is heavier than the right resulting in ball 1 being heavy; and I'll use `=:⌖` to represent the situation where the two sides balance, resulting in no ball being an oddball.

In [17]:
def do(puzzle):
    "Solve the puzzle; verify the solution; and print the solution in a pretty format."
    tree = solve(puzzle)
    assert (tree is None) or valid(tree, puzzle)
    print(pretty_str(tree))
    
def pretty_str(tree, i=0, prefix='') -> str:
    "Pretty, indented string representing a strategy tree."
    if tree is None:
        return 'None'
    elif tree == 0:
        return f'{prefix}⌖'
    elif isinstance(tree, Leaf):
        return f'{prefix}{"+" if tree > 0 else "-"}{ball_str(abs(tree))}'
    else:
        subtrees = (pretty_str(tree.gt, i+1, '>:'), 
                    pretty_str(tree.eq, i+1, '=:'), 
                    pretty_str(tree.lt, i+1, '<:'))
        indent   = ('' if i == 0 else ('\n' + '  ' * i))
        def side(balls): return ''.join(map(ball_str, sorted(balls)))
        return f'{indent}{prefix}{side(tree.L)} ⚖ {side(tree.R)} ➔ {" ".join(subtrees)}'
    
def ball_str(i) -> str:
    """Character string representing ball number `i`."""
    balls =' ①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖㉗㉘㉙㉚㉛㉜㉝㉞㉟㊱㊲㊳㊴㊵㊶㊷㊸㊹㊺㊻㊼㊽㊾㊿'
    return balls[i] if (1 <= i < len(balls)) else str(i) + ' '

In [18]:
# The easier puzzle from before: 3 balls, 1 weighing, only heavier balls possible
do(Puzzle(3, 1, {gt}))

① ⚖ ③ ➔ >:+① =:+② <:+③


In [19]:
# 3 balls, 2 weighings, lighter or heavier balls possible
do(Puzzle(3, 2, {lt, gt}))

① ⚖ ③ ➔ 
  >:② ⚖ ① ➔ >:⌖ =:-③ <:+① 
  =:① ⚖ ② ➔ >:-② =:⌖ <:+② 
  <:① ⚖ ② ➔ >:⌖ =:+③ <:-①


In [20]:
# The original puzzle with 12 balls and 3 weighings
do(p12)

①②③④ ⚖ ⑨⑩⑪⑫ ➔ 
  >:①⑦⑩⑫ ⚖ ②⑥⑧⑨ ➔ 
    >:① ⚖ ⑫ ➔ >:+① =:-⑨ <:⌖ 
    =:①⑦⑫ ⚖ ④⑤⑪ ➔ >:-⑪ =:+③ <:+④ 
    <:①⑦⑪⑫ ⚖ ③⑤⑥⑩ ➔ >:-⑩ =:+② <:-⑫ 
  =:②⑦⑩ ⚖ ①⑥⑧ ➔ 
    >:⑥⑦⑨ ⚖ ②⑤⑪ ➔ >:+⑦ =:-⑧ <:-⑥ 
    =:①②③④⑤ ⚖ ⑧⑨⑩⑪⑫ ➔ >:+⑤ =:⌖ <:-⑤ 
    <:④⑤⑧⑨⑫ ⚖ ①②③⑥⑩ ➔ >:+⑧ =:-⑦ <:+⑥ 
  <:①⑩⑫ ⚖ ③⑧⑨ ➔ 
    >:⑫ ⚖ ⑩ ➔ >:+⑫ =:-③ <:+⑩ 
    =:④⑦⑧ ⚖ ②③⑤ ➔ >:-② =:+⑪ <:-④ 
    <:① ⚖ ⑫ ➔ >:⌖ =:+⑨ <:-①


In [21]:
# A different random solution to the same puzzle
do(p12)

①②③④ ⚖ ⑨⑩⑪⑫ ➔ 
  >:③⑩⑪ ⚖ ④⑦⑨ ➔ 
    >:①②③ ⚖ ⑩⑪⑫ ➔ >:+③ =:-⑨ <:⌖ 
    =:②⑫ ⚖ ③⑧ ➔ >:+② =:+① <:-⑫ 
    <:④⑥⑧⑩ ⚖ ①⑤⑨⑫ ➔ >:+④ =:-⑪ <:-⑩ 
  =:③⑤⑧⑪ ⚖ ①②⑥⑨ ➔ 
    >:①②③④⑤ ⚖ ⑧⑨⑩⑪⑫ ➔ >:+⑤ =:-⑥ <:+⑧ 
    =:①②③④⑤⑥ ⚖ ⑦⑧⑨⑩⑪⑫ ➔ >:-⑦ =:⌖ <:+⑦ 
    <:①②③④⑤ ⚖ ⑧⑨⑩⑪⑫ ➔ >:-⑧ =:+⑥ <:-⑤ 
  <:②④⑧⑫ ⚖ ①③⑤⑦ ➔ 
    >:③⑩ ⚖ ①⑥ ➔ >:-① =:+⑫ <:-③ 
    =:②⑩ ⚖ ⑦⑪ ➔ >:+⑩ =:+⑨ <:+⑪ 
    <:①② ⚖ ⑪⑫ ➔ >:⌖ =:-④ <:-②


I note that the traditional answer to the 12-ball puzzle weighs 4 balls on each side of the first weighing, three balls on the second weighing, and 2 balls on the third weighing. But my program is not so orderly. It always weighs 4 balls on the first weighing (because that is the only good partition), but from there it can vary; it won't necessarily minimize the number of balls on each side of the scale, nor make one branch of the tree be symmetric with another branch.

# Other Puzzles with 3 Weighings



We can do **12 balls in 3 weighings** even when we add in the possibility that there is no oddball–all the balls weigh the same:

In [22]:
do(Puzzle(12, 3, oddballs={lt, gt, 0}))

①②③④ ⚖ ⑨⑩⑪⑫ ➔ 
  >:①⑥⑫ ⚖ ②④⑨ ➔ 
    >:① ⚖ ⑫ ➔ >:+① =:-⑨ <:⌖ 
    =:③⑪ ⚖ ⑧⑨ ➔ >:+③ =:-⑩ <:-⑪ 
    <:③⑤⑦⑧⑨ ⚖ ①④⑥⑩⑫ ➔ >:-⑫ =:+② <:+④ 
  =:①⑥⑧ ⚖ ③⑦⑩ ➔ 
    >:⑧ ⚖ ⑥ ➔ >:+⑧ =:-⑦ <:+⑥ 
    =:①②③④⑤ ⚖ ⑧⑨⑩⑪⑫ ➔ >:+⑤ =:⌖ <:-⑤ 
    <:④⑦⑧⑪ ⚖ ①③⑤⑨ ➔ >:+⑦ =:-⑥ <:-⑧ 
  <:③⑥⑧⑪ ⚖ ①④⑤⑨ ➔ 
    >:②④⑦ ⚖ ①③⑥ ➔ >:-① =:+⑪ <:-④ 
    =:②④⑨⑩⑪ ⚖ ③⑤⑥⑦⑧ ➔ >:+⑩ =:+⑫ <:-② 
    <:①②③ ⚖ ⑩⑪⑫ ➔ >:⌖ =:+⑨ <:-③


Can we solve the **13-balls in 3 weighings** puzzle, which has 26 possibilities? After all, 26 is less than 3<sup>3</sup> = 27.

In [23]:
p13 = Puzzle(13, 3)
do(p13)

None


**No**, and the reason is that there is no first weighing that partitions the 26 possibilities into three branches, each of which has 9 or fewer possibilities (and thus can be handled in the two remaining weighings). No matter what we do, there will always be a branch with 10 or more possibilities:

In [24]:
partition([1, 2, 3, 4], [10, 11, 12, 13], p13.oddballs)

{'gt': {-13, -12, -11, -10, 1, 2, 3, 4},
 'eq': {-9, -8, -7, -6, -5, 5, 6, 7, 8, 9},
 'lt': {-4, -3, -2, -1, 10, 11, 12, 13}}

In [25]:
partition([1, 2, 3, 4, 5], [9, 10, 11, 12, 13], p13.oddballs)

{'gt': {-13, -12, -11, -10, -9, 1, 2, 3, 4, 5},
 'eq': {-8, -7, -6, 6, 7, 8},
 'lt': {-5, -4, -3, -2, -1, 9, 10, 11, 12, 13}}

I'll define `partition_sizes` to make this more clear:

In [26]:
def partition_sizes(puzzle, B) -> List[int]:
    "How big is each branch of a partition with B balls on both sides?"
    part = partition(puzzle.balls[:B], puzzle.balls[-B:], puzzle.oddballs)
    return [len(oddballs) for oddballs in part.values()]

In [27]:
{B: partition_sizes(p13, B) for B in range(2, 7)}

{2: [4, 18, 4], 3: [6, 14, 6], 4: [8, 10, 8], 5: [10, 6, 10], 6: [12, 2, 12]}

We can do **27 balls in 3 weighings** if we know that the oddball can only be lighter, not heavier:

In [28]:
do(Puzzle(27, 3, {lt}))

①②③④⑤⑥⑦⑧⑨ ⚖ ⑲⑳㉑㉒㉓㉔㉕㉖㉗ ➔ 
  >:③⑪⑯⑲⑳㉖ ⚖ ①⑨⑱㉒㉓㉕ ➔ 
    >:⑲㉒㉔ ⚖ ⑦㉑㉕ ➔ >:-㉕ =:-㉓ <:-㉒ 
    =:⑲㉔ ⚖ ⑩㉑ ➔ >:-㉑ =:-㉗ <:-㉔ 
    <:⑥⑨⑫⑮⑲㉔㉗ ⚖ ①④⑦⑩⑱㉑㉖ ➔ >:-㉖ =:-⑳ <:-⑲ 
  =:①②③④⑤⑥⑦⑧⑨⑩⑪⑫ ⚖ ⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖㉗ ➔ 
    >:①②③⑤⑨⑮⑱⑲㉑㉒㉕㉗ ⚖ ⑥⑦⑧⑩⑪⑫⑭⑰⑳㉓㉔㉖ ➔ >:-⑰ =:-⑯ <:-⑱ 
    =:①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬ ⚖ ⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖㉗ ➔ >:-⑮ =:-⑭ <:-⑬ 
    <:②③⑧⑫⑬㉕㉗ ⚖ ④⑤⑪⑭⑲㉑㉖ ➔ >:-⑪ =:-⑩ <:-⑫ 
  <:①⑤⑦⑪㉔㉗ ⚖ ③④⑧⑮⑲㉖ ➔ 
    >:④⑤⑩⑮ ⚖ ①⑧⑯㉒ ➔ >:-⑧ =:-③ <:-④ 
    =:②⑤⑧⑪⑫⑭⑮㉔ ⚖ ①④⑨⑰⑱⑳㉒㉓ ➔ >:-⑨ =:-⑥ <:-② 
    <:①②⑧⑩⑫⑮㉑㉔㉖ ⚖ ④⑤⑥⑪⑯⑱㉒㉓㉗ ➔ >:-⑤ =:-⑦ <:-①


And we can do **26 balls** under the condition that either one ball is lighter or all the balls weigh the same:

In [29]:
do(Puzzle(26, 3, {lt, eq}))

①②③④⑤⑥⑦⑧⑨ ⚖ ⑱⑲⑳㉑㉒㉓㉔㉕㉖ ➔ 
  >:③⑤⑨⑰⑲㉓㉔ ⚖ ④⑥⑩⑯⑱㉒㉖ ➔ 
    >:①⑬⑰㉖ ⚖ ⑤⑥⑫⑱ ➔ >:-⑱ =:-㉒ <:-㉖ 
    =:⑭⑮⑱㉒㉔㉕ ⚖ ①④⑬⑰⑳㉓ ➔ >:-⑳ =:-㉑ <:-㉕ 
    <:②⑤⑥⑧⑨⑩⑯㉓ ⚖ ①③⑪⑮⑲㉑㉕㉖ ➔ >:-⑲ =:-㉔ <:-㉓ 
  =:①②③④⑤⑥⑦⑧⑨⑩⑪⑫ ⚖ ⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖ ➔ 
    >:①③⑤⑦⑮⑱㉖ ⚖ ④⑥⑧⑩⑪⑯㉔ ➔ >:-⑯ =:-⑰ <:-⑮ 
    =:①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬ ⚖ ⑭⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖ ➔ >:-⑭ =:⌖ <:-⑬ 
    <:①⑤⑦⑧⑪⑯⑱㉓㉔㉕ ⚖ ②④⑥⑨⑫⑭⑮⑳㉒㉖ ➔ >:-⑫ =:-⑩ <:-⑪ 
  <:①②⑤⑫㉑㉒㉓ ⚖ ③④⑥⑩⑲⑳㉖ ➔ 
    >:⑥⑭ ⚖ ④⑱ ➔ >:-④ =:-③ <:-⑥ 
    =:④⑤⑧⑫⑰⑳㉓㉔ ⚖ ①②③⑦⑩⑪⑭㉕ ➔ >:-⑦ =:-⑨ <:-⑧ 
    <:②③⑦⑫⑬⑳㉑㉔ ⚖ ④⑤⑥⑧⑰⑲㉒㉕ ➔ >:-⑤ =:-① <:-②


Here's a puzzle with **26 balls** where either one of the odd-numbered balls is heavier, or one of the even-numbered balls is lighter, or there is no oddball.

In [30]:
N = 26
odd_even = [(+b if b % 2 else -b) for b in range(N + 1)]
p26 = Puzzle(N, 3, oddballs=set(odd_even))
print(odd_even)

[0, 1, -2, 3, -4, 5, -6, 7, -8, 9, -10, 11, -12, 13, -14, 15, -16, 17, -18, 19, -20, 21, -22, 23, -24, 25, -26]


In [31]:
do(p26)

④⑥⑪⑫⑭⑮⑰⑳㉕ ⚖ ①②③⑨⑩㉒㉓㉔㉖ ➔ 
  >:①⑥⑨⑩⑬⑮⑯⑱㉑㉒ ⚖ ②③④⑤⑦⑧⑫⑳㉔㉕ ➔ 
    >:①②③ ⚖ ㉔㉕㉖ ➔ >:-㉔ =:+⑮ <:-② 
    =:①②③④⑤⑥⑦⑧⑨⑩ ⚖ ⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖ ➔ >:-㉖ =:+⑪ <:+⑰ 
    <:①②③④⑤ ⚖ ㉒㉓㉔㉕㉖ ➔ >:-㉒ =:-⑩ <:+㉕ 
  =:①②③④⑤⑥⑦⑧⑨ ⚖ ⑱⑲⑳㉑㉒㉓㉔㉕㉖ ➔ 
    >:①③⑥⑦⑨⑯⑱⑲㉖ ⚖ ⑩⑫⑬⑭⑰⑳㉑㉔㉕ ➔ >:+⑦ =:+⑤ <:-⑱ 
    =:①⑦⑩⑪⑰⑱⑳㉑㉓㉕㉖ ⚖ ②③⑤⑥⑧⑨⑬⑭⑮⑯㉔ ➔ >:-⑯ =:⌖ <:+⑬ 
    <:①⑥⑧⑨⑰⑱㉑ ⚖ ②③⑤⑦⑩⑭㉖ ➔ >:+㉑ =:+⑲ <:-⑧ 
  <:①②③④⑤⑥⑦ ⚖ ⑳㉑㉒㉓㉔㉕㉖ ➔ 
    >:②③④⑧⑫⑮⑳㉔㉕ ⚖ ⑤⑥⑩⑪⑬⑱㉑㉒㉓ ➔ >:+③ =:+① <:-⑳ 
    =:①②③④⑤⑥⑦⑧⑨⑩⑪⑫ ⚖ ⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖ ➔ >:+⑨ =:-⑭ <:-⑫ 
    <:⑦⑧⑨⑮⑯⑰⑱㉔㉕㉖ ⚖ ④⑤⑩⑫⑭⑲⑳㉑㉒㉓ ➔ >:-④ =:-⑥ <:+㉓


Another variation: **27 balls, 3 weighings**, the oddball can only be heavier, but one ball (number 27) is poisonous and can't be touched or placed on the balance scale. It might, however be the heavier ball, and you still need to report it as such if it is. 

We describe this situation by defining a puzzle with 26 (weighable) balls, but with the oddballs including all the positive ball numbers, plus +27. Note that when all three weighings are equal, the strategy tree correctly reports `+㉗`.

In [32]:
do(Puzzle(26, 3, oddballs={gt, 27}))

①②③④⑤⑥⑦⑧⑨ ⚖ ⑱⑲⑳㉑㉒㉓㉔㉕㉖ ➔ 
  >:③④⑦⑩⑯㉒㉔㉕㉖ ⚖ ②⑧⑨⑫⑬⑰⑱⑲㉓ ➔ 
    >:③⑭⑯⑰⑱㉒㉖ ⚖ ⑥⑦⑪⑲⑳㉑㉕ ➔ >:+③ =:+④ <:+⑦ 
    =:①③⑪⑬⑭⑮㉒ ⚖ ⑥⑧⑨⑩⑫㉑㉕ ➔ >:+① =:+⑤ <:+⑥ 
    <:①③⑧⑫⑬⑰㉑ ⚖ ②⑥⑦⑭⑱⑲㉕ ➔ >:+⑧ =:+⑨ <:+② 
  =:①②③④⑤⑥⑦⑧⑨⑩⑪⑫ ⚖ ⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖ ➔ 
    >:①②③⑤⑥⑩⑭⑯⑰⑲㉑ ⚖ ④⑦⑧⑨⑪⑬⑮⑱⑳㉓㉕ ➔ >:+⑩ =:+⑫ <:+⑪ 
    =:①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬ ⚖ ⑭⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖ ➔ >:+⑬ =:+㉗ <:+⑭ 
    <:⑦⑩⑮⑲㉖ ⚖ ③⑤⑪⑬⑯ ➔ >:+⑮ =:+⑰ <:+⑯ 
  <:②⑩⑳㉓㉔ ⚖ ⑧⑪⑱⑲㉒ ➔ 
    >:⑩⑫⑬⑱㉔ ⚖ ①⑮⑲⑳㉒ ➔ >:+㉔ =:+㉓ <:+⑳ 
    =:④⑦⑪⑮⑱⑳㉑㉓ ⚖ ①②⑤⑬⑭⑲㉔㉕ ➔ >:+㉑ =:+㉖ <:+㉕ 
    <:⑨⑩⑪⑱ ⚖ ③④⑤㉒ ➔ >:+⑱ =:+⑲ <:+㉒


# Puzzles with 4 Weighings

We can tackle larger puzzles. With 4 weighings, we can theoretically handle up to 3<sup>4</sup> = 81 possibilities. Can we solve for **40 balls**, heavy or light?

In [33]:
p40 = Puzzle(40, 4)

do(p40)

None


Unfortunately, **no**, we didn't find a solution. It is because the `partition_sizes` always leaves us with a branch of size 28 or more, which can;t be handled in the three remaining weighings:

In [34]:
partition_sizes(p40, 13)

[26, 28, 26]

In [35]:
partition_sizes(p40, 14)

[28, 24, 28]

How about **39 balls, 4 weighings**, heavier or lighter, with the possibility that no ball is odd? That's 39 &times; 2 + 1 = 79 possibilities, which is less than 81. 

In [36]:
p39 = Puzzle(39, 4, {lt, eq, gt})

In [37]:
partition_sizes(p39, 13)

[26, 27, 26]

In [38]:
do(p39)

①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬ ⚖ ㉗㉘㉙㉚㉛㉜㉝㉞㉟㊱㊲㊳㊴ ➔ 
  >:⑦⑧⑪⑫⑮㉑㉓㉖㉙㉞㊲㊴ ⚖ ②⑥⑨⑬⑭⑰㉒㉘㉛㉜㉟㊱ ➔ 
    >:①②③⑤⑦⑪⑮⑯⑱㉙㉚㉛㉟ ⚖ ⑧⑩⑬⑭⑰⑲㉑㉒㉓㉕㉖㉘㉞ ➔ 
      >:①④⑪⑬⑭⑯㉑㉒㉔㉖㉗㉘㉙㉚㉜㊲ ⚖ ③⑤⑥⑧⑩⑫⑮⑱⑳㉓㉕㉛㉞㊱㊳㊴ ➔ >:+⑪ =:+⑦ <:-㉘ 
      =:①③⑥⑧⑯⑲㉒㉔㉜㉞㊴ ⚖ ②⑤⑦⑨⑮⑰㉑㉓㉖㊱㊲ ➔ >:-㊱ =:+⑫ <:-㉜ 
      <:①②④⑪⑳㉒㉖㉗㉘㉚ ⚖ ⑥⑦⑧⑩⑮⑰㉕㉝㉟㊳ ➔ >:-㉟ =:-㉛ <:+⑧ 
    =:④⑥⑪⑮㉓㉔㉙㉝㉞ ⚖ ③⑤⑦⑧⑬⑯㉗㊱㊳ ➔ 
      >:⑤⑦⑧⑩⑪㉗㉚㉛㉜㉝㉞㊱㊲ ⚖ ⑨⑫⑬⑯⑰⑱㉒㉓㉔㉕㉖㉟㊳ ➔ >:-㊳ =:+④ <:-㉗ 
      =:③⑤⑥⑧⑨⑬⑮⑱㉖㉗㉝㉞㉟ ⚖ ②⑦⑩⑪⑭⑯⑳㉑㉘㉚㉛㉜㊴ ➔ >:-㉚ =:+① <:+⑩ 
      <:①⑦⑫⑱⑳㉒㉔㉕㉗㉙㊱ ⚖ ②③⑩⑮⑯⑰㉑㉛㉜㉝㉟ ➔ >:-㉝ =:+⑤ <:+③ 
    <:④⑤⑦⑨⑩⑮⑯㉔㉖㉚㉟㊱㊲ ⚖ ①②③⑧⑫⑰⑲㉑㉘㉙㉛㉝㉞ ➔ 
      >:⑦⑬⑲㉔㉜㊲ ⚖ ⑨㉓㉕㉗㉙㉚ ➔ >:-㉙ =:-㉞ <:+⑨ 
      =:①③⑤⑧⑨⑫⑭⑲㉘㉛㉞㊲㊳ ⚖ ⑥⑦⑪⑮⑯⑳㉒㉓㉔㉙㉝㉟㊴ ➔ >:-㊴ =:+⑬ <:+⑥ 
      <:①② ⚖ ㊳㊴ ➔ >:+② =:-㊲ <:⌖ 
  =:①②⑤⑥⑦⑧⑨⑪⑫⑮⑰㉖㉞㊳㊴ ⚖ ④⑩⑬⑱⑲㉒㉓㉔㉕㉗㉘㉙㉚㉝㊱ ➔ 
    >:④⑤⑥⑨⑩⑭⑮⑱⑲㉒㉚㉛㊳ ⚖ ①②⑬⑳㉑㉓㉕㉗㉝㉞㉟㊱㊴ ➔ 
      >:②④⑨⑭⑲㉒㉔㉖㉝㉟㊴ ⚖ ⑦⑧⑩⑫⑮⑯㉓㉗㉚㊱㊲ ➔ >:-㉓ =:-㉕ <:+⑮ 
      =:①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯ ⚖ ㉔㉕㉖㉗㉘㉙㉚㉛㉜㉝㉞㉟㊱㊲㊳㊴ ➔ >:-㉔ =:+⑰ <:+㉖ 
      <:①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱ ⚖ ㉒㉓㉔㉕㉖㉗㉘㉙㉚㉛㉜㉝㉞㉟㊱㊲㊳㊴ ➔ >:-㉒ =:-⑲ <:-⑱ 
    =:①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲ ⚖ ㉑㉒㉓㉔㉕㉖㉗㉘㉙㉚㉛㉜㉝㉞㉟㊱㊲㊳㊴ ➔ 
      >:③⑥⑨⑫⑳㉓㉕㉙㉚㉟㊱㊳ ⚖ ④⑮⑯㉑㉔㉗㉛㉜㉝㉞㊲㊴ ➔ >:-㉑ =:+⑭ <:+⑯ 

How about **80 balls, 4 weighings** under the condition that no ball can be heavier (thus, 81 possibilities, the maximum)? (Unfortunately, the output gets ugly, because we ran out of unicode circled-number characters, and have to use just integers.)

In [39]:
do(Puzzle(80, 4, {lt, eq}))

①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖㉗ ⚖ 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80  ➔ 
  >:①②④⑤⑥⑱⑲㉔㉕㉗㉚㉜㉟51 55 57 60 61 62 63 65 70 71  ⚖ ⑦⑧⑩⑭⑮㉒㉖㊱㊳㊷㊺㊻㊿53 56 59 64 66 72 73 74 78 79  ➔ 
    >:②⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑳㉗㉘㉙㉚㉛㉜㉞㊱㊼55 57 58 60 61 62 63 66 72 74 76  ⚖ ①④⑤⑥⑲㉑㉒㉓㉔㉕㉝㉟㊲㊳㊵㊶㊷㊸㊹㊺㊻51 52 53 56 59 65 68 69 70 71 73 77 80  ➔ 
      >:③④⑨⑩⑪⑫⑬⑮⑲⑳㉑㉕㉖㉘㉚㉛㉜㉞㊲㊵㊶㊹㊾㊿52 54 56 57 61 62 64 72 75 76 77 78  ⚖ ①②⑤⑥⑦⑧⑭⑯⑰⑱㉒㉓㉔㉗㉙㉝㊱㊳㊷㊸㊺㊼㊽53 55 63 65 66 67 68 69 70 73 74 79 80  ➔ >:-73  =:-59  <:-56  
      =:③④⑤⑧⑬⑭⑮㉑㉒㉓㉖㉘㉚㊱㊺㊻㊿53 54 60 61 62 63 64 66 75 77  ⚖ ②⑥⑦⑩⑫⑳㉗㉙㉛㉜㉝㉞㊶㊸㊹㊽52 55 59 65 68 69 71 74 76 79 80  ➔ >:-79  =:-78  <:-64  
      <:⑥⑨⑲㉓㉛㊱㊲㊹㊺㊿51 63 66 70 73 76  ⚖ ②⑬㉖㉗㉘㉟㊸㊼㊽57 60 61 65 68 74 75  ➔ >:-74  =:-72  <:-66  
    =:④⑥⑧⑨⑪⑱㉖㉗㉞㊱㊵㊷㊸㊼㊾57 58 64 67 69 70 74 79  ⚖ ⑦⑮⑰㉑㉒㉔㉕㉙㉚㉛㉜㉟㊳㊻51 54 55 59 60 61 63 68 80  ➔ 
      >:③⑳㉖54 62  ⚖ ②⑰㉓㉔68  ➔ >:-68  =:-80  <:-54  
      =:②③⑦⑮⑲⑳㉖㉘㉙㉛㉜㉝㊱㊲㊶㊼51 53 55 57 59 67 69 73 76 79  ⚖ ④⑩⑪⑬⑭⑰㉒㉓㉕㊳㊵㊷㊽56 60 61 62 63 64 65 66 70 74 75 78 80  ➔ >:-75  =:-77  <:-7

Looking good (except that the output is longer than the line length, and so is hard to read).

# Puzzles with 5 Weighings

With 5 weighings, 3<sup>5</sup> = 243, so I might be able to handle up to 121 lighter-or-heavier balls (242 possibilities):

In [40]:
p5 = Puzzle(121, 5)
do(p5)

None


Nope, that didn't work. Let's check the partition sizes (they need to be 81 or less to solve in 4 more weighings):

In [41]:
partition_sizes(p5, 40)

[80, 82, 80]

In [42]:
partition_sizes(p5, 41)

[82, 78, 82]

Let's back off to **120 balls, 5 weighings** for 2 &times; 120 + 1 = 241 possibilities:

In [43]:
do(Puzzle(120, 5, {lt, gt, 0}))

①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖㉗㉘㉙㉚㉛㉜㉝㉞㉟㊱㊲㊳㊴㊵ ⚖ 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120  ➔ 
  >:①②③④⑦⑨⑬⑮⑱㉓㉕㉗㉚㊱㊴㊶㊻㊼㊽㊾53 55 57 59 60 65 66 67 70 72 76 81 82 84 85 98 109 111 113 118 120  ⚖ ⑩⑫⑭⑯⑰⑲⑳㉑㉘㉛㉜㉝㉞㉟㊲㊳㊵56 58 61 62 63 68 69 71 73 74 75 80 83 87 91 92 99 103 106 107 110 112 114 119  ➔ 
    >:①⑨⑯㉒㉕㉗㉙㉚㉞㊷㊹㊺㊾51 52 53 57 59 60 61 66 67 68 69 70 72 79 84 86 87 92 97 100 101 107 110 113 116 118  ⚖ ③④⑤⑧⑩⑫⑱⑲⑳㉑㉓㉔㉟㊱㊲㊳㊵㊶㊸㊻㊿54 56 65 71 73 74 75 77 78 82 83 88 93 95 99 105 106 119  ➔ 
      >:①⑦⑪⑬㉓㉕㉖㉛㉜㉝㊳㊸51 58 59 68 73 78 79 81 83 89 90 96 99 103 104 109 111 117 118 119 120  ⚖ ③④⑥⑲㉑㉔㉘㉟㊲㊶㊷㊹㊻㊾㊿52 55 65 66 70 74 77 95 98 100 101 106 107 108 110 112 113 116  ➔ 
        >:①㊶106  ⚖ ㉛93 101  ➔ >:+① =:+㉕ <:-106  
        =:②㉕㉙㉚㊿59 62 71 89 96  ⚖ ④⑨㊺52 77 82 90 98 99 114  ➔ >:+㉚ =:+㉗ <:+⑨ 
        <:①⑥⑨⑫⑳㉑㉔㉕㉗㉘㉛㉟㊵㊷㊸㊹㊺㊻㊾㊿52 57 58 60 61 62 63 67 70 71 76 77 78 81 83 85 86 90 92 93 96 98 100 101 102 103 104 106 107 10

That works.

Or, I could solve a puzzle with 243 possibilites (the maximum possible with 5 weighings): **242 balls**, lighter-only, plus the possibility that no ball is odd:

In [44]:
do(Puzzle(242, 5, {lt, 0}))

①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖㉗㉘㉙㉚㉛㉜㉝㉞㉟㊱㊲㊳㊴㊵㊶㊷㊸㊹㊺㊻㊼㊽㊾㊿51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81  ⚖ 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242  ➔ 
  >:①③⑦⑧⑪⑬⑯⑳㉑㉘㉚㉜㉞㊴㊵㊸㊻㊿51 58 59 60 66 67 68 78 82 87 97 98 103 107 110 111 113 118 122 123 124 125 126 129 132 140 143 144 146 150 155 156 157 159 160 165 166 169 171 172 173 174 175 177 178 181 188 195 197 204 206 207 209 211 216 220 221 222 229 235 240 241  ⚖ ②④⑨⑩⑮⑰⑱⑲㉓㉔㉕㉖㉗㉙㉛㊳㊾56 62 63 65 69 70 71 75 77 79 80 83 85 89 90 93 96 100 102 105 112 115 116 117 121 127 135 136 137 138 141 149 152 153 158 161 162 163 164 176 179 183 187 189 193 194 198 199 202 203 214 215 219 223 224 225 227 228 232 236 237 238 242  ➔ 
    >:①②⑨

# What's Next?

- What other puzzles can you solve?
- Can you make a table summarizing the space of solved and unsolved puzzles? 
- Can you *prove* the unsolved puzzles are unsolvable? 
- Currently, when `solve` returns `None`, it means "no solution was found, but there's no proof that a solution doesn't exist." Can you modify `solve` to return a result that indicates there is no possible solution? (Maybe you can only do this on some puzzles, not all.)
- What about puzzles where your task is to identify *which* ball is odd, but not *how* it is odd? That is, if you get the possibilities down to `{-3, +3}` then you're done; you know ball 3 is the odd one, and you don't care if it is heavy or light.
- Can you find trees that minimize the *mean* number of weighings, rather than minimizing the *maximum* number of weighings?
- What happens when it is a possibility that *two* balls are odd?  Or more?
- What if you had two or more balance scales that you could use in parallel, and the goal was to minimize the number of weighing time periods, not the total number of weighings?
- What else can you discover?